In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd

In [ ]:
# Load dataset
dataset = "results/arp_all_vec.csv"
data_list = pd.read_csv(dataset).to_numpy()
data_list = np.array([np.array([float(x) for x in row[0].split(" ")]) for row in data_list])
print("Dataset size: ", data_list.shape)

In [ ]:
# Set hyperparameters
train_encoding = 200000 # number of packets to train packet feature encoding on
train_clusters = 500000 # number of packets to train clustering module on
W_seg = 50 # framing length 
C = 10 # adjustment in log transform
num_clusters = 50 # number of clusters

In [ ]:
# Get feature encodings using PCA
train_encoding_data = data_list[:train_encoding]
pca = PCA(n_components=20)
pca.fit(train_encoding_data)


In [ ]:
# Transform the remaining data using the learned PCA
remaining_data = data_list[train_encoding:]
embedded_data = pca.transform(remaining_data)

In [ ]:
# Divide packet stream into frames
n_packets = embedded_data.shape[0]
n_frames = n_packets // W_seg
print("Number of packets: ", n_packets)
print("Number of frames: ", n_frames)

In [ ]:
# Perform DFT on each frame and calculate the modulus
modulus_dft = []
for i in range(n_frames):
    frame = embedded_data[i*W_seg:(i+1)*W_seg]
    dft_output = np.fft.fft(frame)
    modulus_output = np.abs(dft_output)
    modulus_dft.append(modulus_output)
modulus_dft = np.array(modulus_dft)

print("Modulus DFT shape: ", modulus_dft.shape)

In [ ]:
# Apply log transform to modulus of DFT outputs
log_modulus_dft = np.log2(modulus_dft + np.ones(modulus_dft.shape))/C

In [ ]:
# Flatten frequency domain features
train_ind = train_clusters // W_seg
train_data = log_modulus_dft[:train_ind,:,].reshape(train_ind, -1)
print(train_data.shape)
test_data = log_modulus_dft[train_ind:,:,].reshape(n_frames - train_ind, -1)
print(test_data.shape)
all_data = log_modulus_dft.reshape(n_frames, -1)
print(all_data.shape)

# Fit KMeans clustering model on training data
kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
kmeans.fit(train_data)

In [ ]:
def get_l2_distances(dataset, kmeans):
    # Calculate the L2 distance between each data point and its closest cluster center
    closest_cluster_centers = kmeans.cluster_centers_[kmeans.predict(dataset)]
    l2_distances = np.linalg.norm(dataset - closest_cluster_centers, axis=1)
    return l2_distances

In [ ]:
# Plot histogram of l2 distances
plt.hist(get_l2_distances(all_data, kmeans), bins=100)
plt.show()

In [ ]:
# Convert from frames back to packets
num_data_pts = data_list.shape[0]
l2_distances = get_l2_distances(all_data, kmeans)
l2_dist_packet = np.append(np.zeros(train_encoding),np.repeat(l2_distances, W_seg))
print("L2 distances shape: ", l2_dist_packet.shape)


In [ ]:
# Load packet labels for plotting
labels = "ARP_MitM_labels.csv"
with open(labels, 'r') as f:
    reader = csv.reader(f)
    labels_list = list(reader)
labels_list = [int(sublist[-1]) for sublist in labels_list]
labels_list = np.array(labels_list)
print("Labels shape: ", labels_list.shape)

In [ ]:
anomaly_dist = []
normal_dist = []
anomaly_indices = []
for i in range(l2_dist_packet.shape[0]):
    if labels_list[i] == 1:
        anomaly_dist.append(l2_dist_packet[i])
        anomaly_indices.append(i)
    elif i > train_encoding:
        normal_dist.append(l2_dist_packet[i])

normal_dist = np.array(normal_dist)
anomaly_dist = np.array(anomaly_dist)
anomaly_indices = np.array(anomaly_indices)
normal_indices = np.setdiff1d(range(train_encoding, num_data_pts),anomaly_indices, True)


In [ ]:
# Plot L2 distances, seperate by benign/malicious
plt.figure(figsize=(11,5))
x_max = 4.5
plt.xlim(0, x_max)
bin_list = [x_max/200.0 * i for i in range(201)]
    
n, bins, patches = plt.hist(anomaly_dist, bins=bin_list, facecolor='g', label="L2 distances of malicious packets", log=True)
bin_centers = 0.5 * (bins[:-1] + bins[1:])
for c, p in zip(bin_centers, patches):
    plt.setp(p, color='#a32632', alpha=0.7)

n, bins, patches = plt.hist(0.9*normal_dist, bins=bin_list, facecolor='g', label="L2 distances of benign packets", log=True)
bin_centers = 0.5 * (bins[:-1] + bins[1:])
for c, p in zip(bin_centers, patches):
    plt.setp(p, color='#1f9156', alpha=0.7)

plt.xlabel("L2 distance from nearest cluster center", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.legend(reversed(handles), reversed(labels), loc="upper right", fontsize=16)    
plt.show()

In [ ]:
# Calculate AUC of ROC curve
def get_roc_auc(normal_rmses, anomaly_rmses, sample_size=30000):
    normal_rmses = np.random.choice(normal_rmses, sample_size)
    anomaly_rmses = np.random.choice(anomaly_rmses, sample_size)
    roc_auc = 0
    for normal_rmse in normal_rmses:
        for anomaly_rmse in anomaly_rmses:
            if anomaly_rmse > normal_rmse:
                roc_auc += 1
    roc_auc /= (sample_size * sample_size)
    return roc_auc

In [ ]:
get_roc_auc(normal_dist, anomaly_dist)

In [ ]:
# Plot feature distributions seperated by false negative/true negative/true positive

def get_column(i):
    return [float(row[i]) for row in data_list]

def get_feature(i):
    col_i = all_data[:,i]
    expand_i = np.append(np.zeros(train_encoding),np.repeat(col_i, W_seg))
    return list(expand_i)

threshold = 1
cols = [1, 2, 3] 
for col in cols:
    column = get_feature(col)
    column_less = []
    column_greater = []
    column_normal = []
    for i in range(len(normal_indices)):
        try:
            column_normal.append(column[normal_indices[i] - 1])
        except:
            break 
    for i in range(anomaly_indices.shape[0]):
        if anomaly_dist[i] < threshold:
            column_less.append(column[anomaly_indices[i] - 1])
        else:
            column_greater.append(column[anomaly_indices[i] - 1])

    max_i = max(np.amax(column_less), np.amax(column_greater))
    min_i = min(np.amin(column_less), np.amin(column_greater))
    plt.figure(figsize=(16,5))
    plt.xlim(min_i, max_i)

    bin_list = [min_i + (max_i - min_i)/200.0 * i for i in range(201)]

    n, bins, patches = plt.hist(column_greater, bins=bin_list, facecolor='g', label="true positive", log=True)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    for c, p in zip(bin_centers, patches):
        plt.setp(p, color='#a32632', alpha=0.6)

    n, bins, patches = plt.hist(column_normal, bins=bin_list, facecolor='g', label="true negative", log=True)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    for c, p in zip(bin_centers, patches):
        plt.setp(p, color='green', alpha=0.4)

    n, bins, patches = plt.hist(column_less, bins=bin_list, facecolor='g', label="false negative", log=True)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    for c, p in zip(bin_centers, patches):
        plt.setp(p, color='#328da8', alpha=0.8)

    plt.xlabel(f"Frequency Domain Feature {col}", fontsize=15)
    plt.ylabel("Frequency", fontsize=15)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(reversed(handles), reversed(labels), loc="upper left", fontsize=18)
    plt.savefig(f"results/arp_whisper_{col}.pdf", bbox_inches="tight")